# Final project of Mobile Robotique 

Student group number 19 : <br/>
Kyan Achtari <br/>
Louis Gavignet <br/>
Louise Genoud <br/>
Joaquim Silveira <br/>

# Introduction
<br/>
The project of Mobile Robotique aims to combine vision, path planning, global navigation, local navigation, and filtering with a kalman filter to move Thymio to the goal.
<br/><br/>
We first create a map composed of differents elements : black obstacles where the Thymio must never go, a blue triangle on Thymio to get his position and orientation in real time, a red square goal, and white small obstacles invisible to the camera.  The camera, using the vision modulus, gets the coordinates of every elements on the map from which we extracts a path. We then use the path planning with Dijkstra to extract the best suited path. 
<br/><br/>
While following the path, Thymio may enconters obstacles not detected by the camera. That's why it constantly scans the near environment with the IR sensors, when one is triggered Thymio avoids the obstacles using local navigation.
<br/><br/>
The robot also use a filter to estimate the position with the help of sensors (camera and motor speed) and to correct it. We choose to implement the kalman filter, which shows a strong efficiency even when the camera is blocked.

# Implementaion with vision

### Hardware and Software Setup

For the computer vision implementation, we will be using the camera METTRE MODEL. The OpenCV library will be used to process and treat the images to our convenience.

### Object recognition

The first step is to differentiate three types of objects: obstacles, the robot and the goal area. They are respectively black, blue and red, can be either squares or triangles, and be placed almost anywhere inside the frame, so long as the obstacles are twice the robot's size a part. The object recognition process for either type consists of the same 4 steps : 

**1. Convert Image to HSV Color Space** : from research, we found out that the Hue Saturation Value color space is ideal for edge detection, as the distinction between the colors is more evident. Also, it is less sensible to the environment's lightning, as a full Value, equivalent to brightness, corresponds to how the color behaves and appears under a certain lightning. Therefore, full brightness does not mean the color will be white, but it's own color will be more radiant.

**2. Smooth the Image**: we smooth the image to reduce noise and make the pixel values inside an object more homogeneous, making it easer to establish an upper and lower color bound (in HSV of course). To smooth the image, we chose to employ the Bilateral Filter proposed by the OpenCV2 library. It is both effective for removing white noise and at the same time keeping edges sharp, which is what we'll need for the next step. Of course, it takes more computing power, but we can compensate this by reducing the image pixel width and height.

**3. Image Mask**: To create the object mask, we take the lower and upper color bounds for it, which were measured using the Gimp image processing software, and use the OpenCv2 InRange function, which basically applies a threshold to the image with the given bound. If the pixel is outside of the bound, it's value is set to 0, else it will be set to 1.

**4. Object Contour and Center Recognition**: Now with the object isolated in the image, we can detect it's contour and center with the OpenCv2 findContours function, which joins pixel with same values(which is why we apply thresholding, as it will work better to detect contours) and that are near by. Of course, it is highly likely that noise is still present up to this stage, which is why we apply an extra step: we approximate the contour with a simpler shape of roughly the same size, calculate it's area and reject it as a valid object if it is less than a certain threshold. This allows us to ignore smaller features that are big enough to not be picked up by the bilateral smoothing, that may be part of the arena texture. By calculating the moment of the approximated contour, we can find both it's area and center point. 

We now have all the informations we need for the objects: it's center, it's area and it's edge points. Each step result is shown by the following images:
| HSV image | Smooth Image | Image Mask | Object Contour|

<img src="images/obstacle/img_processed.png" alt="drawing" width="300"/> <img src="images/obstacle/img_hsv.png" alt="drawing" width="300"/>  <img src="images/obstacle/img_smooth.png" alt="drawing" width="300"/>  
<img src="images/obstacle/mask.png" alt="drawing" width="300"/>  <img src="images/obstacle/frame.png" alt="drawing" width="300"/>
    


### Robot Direction Detection
Unlike with the obstacles and the goal, we also care about the robot's direction, as we need to know it's orientation to correctly move it through the world. For this, we use an isosceles triangle pointing forward. We calculate it's edges and center as mentionned above, and can measure it's forward orienting vector by finding which vertex of it's contour is further away from the center point. This allows us to calculate it's inclination, seen from above, with respect to the x axis, or the camera's horizon. The forward pointing vector is shown in the image below:

<img src="" alt="drawing" width="400"/>

METTRE IMAGE ICI

# Path planning

The vision module returns a list of obstacles, characterized by the coordinates of their vertices. From these points, the path planning module does several things :

- **Obstacle augmentation** : construct a series of points through which the Thymio can go, from the coordinates of the obstacle vertices
- **Graph construction** : build a graph structure with the right connections between vertices (so that Thymio cannot cross an obstacle) and the weights of the edges initialized (they correspond to th distances between the points)
- **Shortest path** : using Dijkstra's algorithm, find the shortest path from the start point to the end point, passing through a subset of the vertices

### Obstacle augmentation
The obstacle augmentation is done by moving each vertex of each obstacle in two different ways, as illustrated in the figure below for vertex A :  

<img src=obstacle.JPG width="400">

The point is moved in the direction parallel to each edge of the obstacle, and then in the perpendicular direction away from the obstacle. The distance of each of these translation is a constant defined as half the Thymio width plus a safety margin. This is done so that the robot will not collide with the corner of the obstacle when going around it.

### Graph construction
The construction of the graph from the augmented vertices is done in 2 steps for each obstacle :
- The consecutive vertices are connected to each other (yellow edges)
- The vertex of the obstacle that is the closest to a vertex of another obstacle is connected to that vertex (red edge)

<img src=path_around_sev.JPG width="800">  

The start and end point are connected to the vertex of the graph that is the closest to them.

```
    while(len(unconnected)!=0):
    dp = distances.copy()[unconnected]
    idx_a = np.argmin(dp,axis=0)[0]
    a = int(dp[idx_a,1])
    remove(a,unconnected)

    for v_b in graph[a]:
        b = v_b[0]

        w_ab = get_weight(a,b,graph)
        if((w_ab > 0) and (distances[b,0]> distances[a,0] + w_ab)):
            distances[b,0] = distances[a,0] + w_ab
            predecessor[b] = a


return distances,predecessor
```

From the resulting list of predecessors, the path (_coord_) can then be extracted by walking through it from end to beginning:

```
coord.append(end)

while(pred[i] != len(pred)-2):
    i = pred[i]
    path_num.append(i)
    coord.append(coordlist[i][0:2])

coord.append(start)
coord.reverse()
```

### Shortest path
The shortest path can be computed using Dijkstra's algorithm on the graph that was constructed. First, the table of _distances_ is initialized with all distances to a value INF representing infinity, and the table of _predecessors_ is created. An _unconnected_ list is also created containing the indexes of all the vertices at the beginning.  

The _graph_ is a list of lists, with every vertex represented by a list of its connected vertices and the weight of the edge connecting them. 
These tables are then updated as such :

# Motion control

### Global navigation

##### Implementation of the movement
We create a fonction called pathing() that uses all the functions needed for moving the Thymio. In every iteration, pathing calls the camera, kalman, and refers to the path given by the path planning. The Thymio moves continuously and actuates the speed of his motors thanks to a proportionnal controller.

##### Rotation 
When the Thymio arrives at an intermediate goal, he has to change his orientation to go to the next one. The function get_angle_between() gives the angular difference between the two vectors

```
def get_angle_between(vec1, vec2):
    vec1_unit = vec1 / np.linalg.norm(vec1)
    vec2_unit = vec2 / np.linalg.norm(vec2)

    return np.arccos(np.dot(vec1_unit, vec2_unit))
```

Then, we make sure the angle belongs to the interval -pi, pi :

```
def wrap_angle(angle):
    if angle > math.pi:
        angle = angle - 2*math.pi
    elif angle < -math.pi:
        angle = angle + 2*math.pi
    return angle
```

##### Controller

We use a proportionnal controller to set the speed of the motors. It takes into account the angle for the correcting or new orientation. We also set an angle minimum under which the robot won't turn, the kalman correction can be very precice so we don't take it in consideration every time. This angle minimum is equal to 0.4 radians, which corresponds to 23 degrees. 

```
def controller(angle)
    if abs(angle) > ANGLE_TOLERANCE:
                speed_l = 80 - kp_rot*(angle)
                speed_r = 80 + kp_rot*(angle)
            else:
                speed_l = SPEED_AVG
                speed_r = SPEED_AVG
    return speed_l, speed_r
```

### Local navigation

We read the 7 horizontal sensor values and store them in prox.vals and multiply them by IR.SCALE.DOWN (which is 0.05), if any of the values is non-zero (other than the last 2, which are behind the Thymio), we enter obstacle avoidance:

Then we multiply the vector prox.vals by LEFT.IR.WEIGHT and RIGHT.IR.WEIGHT for each wheel:

$$
LEFT.IR.WEIGHT =
\left(\begin{array}{cc} 
1 \\
2 \\
-5 \\
-2 \\
-1 \\
0 \\
0 \\
\end{array}\right)
\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;
RIGHT.IR.WEIGHT =
\left(\begin{array}{cc} 
-1 \\
-2 \\
-5 \\
2 \\
1 \\
0 \\
0 \\
\end{array}\right)
\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;
$$

The speed of each wheel is their respective weight multiplied by the sensor values, and + 10

```
if sum(prox_vals[0], prox_vals[1], prox_vals[2], prox_vals[3], prox_vals[4]) > 0:
    obst = True

if obst == True:
    speed_l = sum(prox_vals * LEFT_IR_WEIGHT)  + 10
    speed_r = sum(prox_vals * RIGHT_IR_WEIGHT) + 10
```

# Filtering with Kalman filter 

### Overview and prediction

To calculate the beliefs of the robot position we setup a Bayes filter. It's a recrsive filter taht calculte first a prediction of the robot positioon and then update the measurement. We implemented an extended Kalman filter because we conside a nonlinear gaussian state space model, our state are $X = (x,y, \theta)$. The states are modified by the speed of the left and right motors U = (speed_left, spped_right). The nonlinear modification of each paramters can be modelled like this : 
$$ x_{t+1} = x_{t} + v_{average}\cdot dt\cdot cos(\theta) $$ 
$$ y_{t+1} = y_{t} + v_{average} \cdot dt\cdot sin(\theta) $$ 
$$ \theta_{t+1} = \theta_{t} + dt\cdot \omega $$  

<c/><c/>
We can write the systeme above with some matrices :
$$ X_{t+1} = A\cdot X_{t} + B\cdot U $$

<c/><c/>
With the following matrices A and B :
$$
A = 
\left(\begin{array}{cc} 
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{array}\right)
$$

$$
B = 
\left(\begin{array}{cc} 
0.5\cdot dt\cdot cos(\theta) & 0.5 \cdot dt\cdot cos(\theta)\\
0.5\cdot dt \cdot sin(\theta) & 0.5\cdot dt\cdot sin(\theta) \\
-dt/wheeldist & dt/wheeeldist \\
\end{array}\right)
$$


The state transition covariance matrix Q, contains the variance of the motors speed :

$$ 
Q = 
\left(\begin{array}{cc}
speedvar & 0\\
0 & speedvar\\
\end{array}\right)
$$

<br/>
The covariance of the obervation noise matrix R, errors come from the camera when measuring a known object :

$$ 
R = 
\left(\begin{array}{cc}
camvar & 0 & 0\\
0 & camvar & 0\\
0 & 0 & camanglevar\\
\end{array}\right)
$$

### Prediction

We can predict the next position of the robot thanks to the set of equations of the nonlinear $X_{t+1}$. We calculate the predication every time interval dt, this periode corresponds to actuation time of our robot. Every dt we calculate the new orientation to follow the path, we update the speed of the motors, and we check with the Kalman filter that Thymio is on the right path.

We also compute the prediction of an estimate of the covariance in this function :
$P = P+B\cdot Q\cdot B^T$

### Update 

Update the Kalman filter state estimate previously with the Kalman gain K, and the difference between the actual state z and the predicted one x. In the mean time, we update the values of P.

$$I = z-X$$
and $$K = P \cdot H^T\cdot S$$  with $$S=R+H\cdot P^T\cdot H$$
then $$X=X+K\cdot I$$
and $$P = P-K\cdot H\cdot P$$ 

### Filter

We then implement the filter function by calling the prediction function and the update function in this order every dt